In [10]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [11]:
# Set the file path
filepath = Path("../Resources/googl_data.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,Open,High,Low,Close,Volume,Volatility,sentiment,RSI
0,2019-01-02,1027.20,1060.79,1025.2750,1053.240,1346501,0.279226,0.001863,NaN
1,2019-01-03,1050.67,1066.26,1022.3700,1026.505,1546910,0.297727,0.023873,NaN
2,2019-01-04,1041.63,1080.00,1036.8600,1076.550,1792417,0.498923,0.059965,NaN
3,2019-01-07,1080.97,1082.70,1062.6400,1076.010,1872050,0.073502,0.099050,NaN
4,2019-01-08,1084.77,1093.35,1068.3486,1085.510,1402631,0.007682,0.186789,NaN


In [14]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the short window and long windows
short_window = 5
long_window = 15

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2019-01-02,1053.240,NaN,NaN,0.0,NaN
2019-01-03,1026.505,NaN,NaN,0.0,0.0
2019-01-04,1076.550,NaN,NaN,0.0,0.0
2019-01-07,1076.010,NaN,NaN,0.0,0.0
2019-01-08,1085.510,1063.563,NaN,0.0,0.0
2019-01-09,1083.220,1069.559,NaN,0.0,0.0
2019-01-10,1078.450,1079.948,NaN,0.0,0.0
2019-01-11,1062.930,1077.224,NaN,0.0,0.0
2019-01-14,1052.845,1072.591,NaN,0.0,0.0


In [15]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)